In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/features.txt'

feature_name_df = pd.read_csv(url, sep = '\s+', header = None, 
                              names = ['column_index', 'column_name'])

feature_name = feature_name_df.iloc[:,1].values.tolist()

x_tr_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/train/X_train.txt'
x_t_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/test/X_test.txt'

x_tr = pd.read_csv(x_tr_url, sep ='\s+',header = None)
x_t = pd.read_csv(x_t_url, sep ='\s+',header = None)

x_tr.columns = feature_name
x_t.columns = feature_name

y_tr_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/train/y_train.txt'
y_t_url = 'https://raw.githubusercontent.com/PinkWink/ML_tutorial/master/dataset/HAR_dataset/test/y_test.txt'

y_tr = pd.read_csv(y_tr_url, sep ='\s+',header = None, names= ['action'])
y_t = pd.read_csv(y_t_url, sep ='\s+',header = None, names = ['action'])

In [43]:
x_tr.shape , y_tr.shape

((7352, 561), (7352, 1))

## GBM
- 일반적으로 gbm이 성능 자체는 random forest 보다 낫다고 알려져있음.
- skelarn 의 gbm은 속도가 아주 느린 것으로 알려져있음
- 현재 pc의 경우 25분 이상이 소요됨

In [2]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import time
import warnings
warnings.filterwarnings("ignore")

- import time의 경우 jupyter에서는 start_time = time.time()으로 잡아주지만 여기서는 %%time으로 사용하면 됨.


In [9]:
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0xC0FFEE)
gb_clf.fit(x_tr,y_tr)
gb_pred = gb_clf.predict(x_t)
print(f'aac: {accuracy_score(y_t, gb_pred)}')
print(f'fit time:{time.time()- start_time}')


aac: 0.9382422802850356
fit time:1697.7947993278503


- 아래 코드 40분 넘게 걸림
- 결과만 확인해봄
- GBM에서 accuracy_score(y_t, grid.best_estimator_.predict(x_t)) = 94퍼센트 정도 나옴

In [11]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators': [100,500],
    'learning_rate': [0.05,0.1]

}

start_time = time.time()
grid = GridSearchCV(gb_clf, param_grid=params, cv = 2, verbose= 3, n_jobs = -1)
grid.fit(x_tr,y_tr)

print(f'fit time:{time.time()- start_time}')

Fitting 2 folds for each of 4 candidates, totalling 8 fits


KeyboardInterrupt: 

---
## XGBoost

In [12]:
pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 435.7 kB/s eta 0:04:47
   ---------------------------------------- 0.1/124.9 MB 907.3 kB/s eta 0:02:18
   ---------------------------------------- 0.1/124.9 MB 901.1 kB/s eta 0:02:19
   ---------------------------------------- 0.2/124.9 MB 980.4 kB/s eta 0:02:08
   ---------------------------------------- 0.3/124.9 MB 1.1 MB/s eta 0:01:58
   ---------------------------------------- 0.3/124.9 MB 1.1 MB/s eta 0:01:57
   ---------------------------------------- 0.4/124.9 MB 1.1 MB/s eta 0:01:55
   ---------------------------------------- 0.4/124.9 MB 1.1 MB/s eta 0:01:54
   ---------------------------------------- 0.5/124.9 MB 1.1 MB/s eta 0:01:55
   ---------------------------------------- 0.6/124.9 MB 1.1 MB/s eta 0:01:52
   ---------------------------------------- 0.6/124.9 MB 1.1 MB/s eta 0:01:53
   ---------------------------------------- 0.7/124.9 MB 1.1 MB

In [15]:
y_tr = y_tr - 1  # Shift labels down by one

In [44]:
from xgboost import XGBClassifier

start_time= time.time()
xgb = XGBClassifier(n_estimators=400, learning_rate= 0.1, max_depth=3)
xgb.fit(x_tr.values,y_tr)

print(f'fit time:{time.time()- start_time}')


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0 1 2 3 4 5], got [1 2 3 4 5 6]

In [27]:
accuracy_score(y_t,xgb.predict(x_t.values))

0.024092297251442144

In [40]:
evals = [(x_t.values,y_t.values)]

start_time = time.time()
xbg = XGBClassifier(n_estimators=400, learning_rate= 0.1, max_depth=3)
xgb.fit(x_tr.values, y_tr, eval_set=evals, early_stopping_rounds=10)

print(f'fit_time:{time.time()- start_time}')

TypeError: fit() got an unexpected keyword argument 'early_stopping_rounds'

---
## LGBM

In [39]:
evals

[(array([[ 0.25717778, -0.02328523, -0.01465376, ..., -0.72000927,
           0.27680104, -0.0579783 ],
         [ 0.28602671, -0.01316336, -0.11908252, ..., -0.69809082,
           0.28134292, -0.08389801],
         [ 0.27548482, -0.02605042, -0.11815167, ..., -0.70277146,
           0.28008303, -0.0793462 ],
         ...,
         [ 0.34996609,  0.03007744, -0.11578796, ..., -0.65535684,
           0.27447878,  0.18118355],
         [ 0.23759383,  0.01846687, -0.09649893, ..., -0.65971859,
           0.26478161,  0.18756291],
         [ 0.15362719, -0.01843651, -0.13701846, ..., -0.66008023,
           0.26393619,  0.1881034 ]]),
        action
  0          5
  1          5
  2          5
  3          5
  4          5
  ...      ...
  2942       2
  2943       2
  2944       2
  2945       2
  2946       2
  
  [2947 rows x 1 columns])]

In [36]:
from lightgbm import LGBMClassifier

start_time = time.time()
lgbm = LGBMClassifier(n_estimators=400)
lgbm.fit(x_tr.values, y_tr, eval_set=evals, early_stopping_rounds=20)

TypeError: fit() got an unexpected keyword argument 'early_stopping_rounds'

In [37]:
pip install --upgrade lightgbm

Note: you may need to restart the kernel to use updated packages.


In [38]:
lgbm.fit(x_tr.values, y_tr, eval_set=evals, early_stopping_rounds=20)

TypeError: fit() got an unexpected keyword argument 'early_stopping_rounds'